# Sudoku-5 : Particle Swarm Optimization (PSO)

**Niveau** : Metaheuristique  
**Duree** : ~20 min  
**Prerequis** : Sudoku-0-Environment

## Objectifs d'apprentissage

1. Comprendre les principes de l'optimisation par essaim de particules
2. Adapter le PSO a la resolution de Sudoku
3. Comparer les performances avec les autres metaheuristiques (GA, SA)

## Navigation

| << Precedent | [Index](./README.md) | Suivant >> |
|-------------|---------------------|-----------|
| [Sudoku-4-SimulatedAnnealing](Sudoku-4-SimulatedAnnealing.ipynb) | | [Sudoku-6-AIMA-CSP](Sudoku-6-AIMA-CSP.ipynb) |

## 1. Introduction au PSO

Le **Particle Swarm Optimization** (PSO) est une metaheuristique inspiree du comportement social des oiseaux et des poissons. Developpe par Kennedy et Eberhart en 1995, l'algorithme simule un essaim de particules qui explorent l'espace de recherche.

### Principes fondamentaux

1. **Particules** : Chaque particule represente une solution candidate
2. **Position** : La position de la particule = configuration de la grille
3. **Velocite** : Direction et vitesse de deplacement dans l'espace de recherche
4. **pBest** : Meilleure position personnelle de la particule
5. **gBest** : Meilleure position globale de l'essaim

### Equation de mise a jour

```
v(t+1) = w * v(t) + c1 * r1 * (pBest - x(t)) + c2 * r2 * (gBest - x(t))
x(t+1) = x(t) + v(t+1)
```

Ou :
- `w` : poids d'inertie (impact de la velocite precedente)
- `c1`, `c2` : coefficients d'apprentissage (cognitif et social)
- `r1`, `r2` : nombres aleatoires [0, 1]

### Adaptation au Sudoku

Pour adapter le PSO au Sudoku, nous utilisons une variante specifique :
- **Workers** (90%) : Exploitent leur voisinage et convergent vers `gBest`
- **Explorers** (10%) : Explorent aleatoirement l'espace de recherche
- **Merge** : Fusion des meilleures solutions Worker et Explorer

In [ ]:
#!import Sudoku-0-Environment.ipynb

## 2. Implementation des classes auxiliaires

In [ ]:
// Helper pour la manipulation des matrices Sudoku
public static class MatrixHelperPSO
{
    public const int SIZE = 9;
    public const int BLOCK_SIZE = 3;

    public static int[,] CreateMatrix(int m, int n)
    {
        return new int[m, n];
    }

    public static (int row, int column) Corner(int block)
    {
        int r = (block / 3) * 3;
        int c = (block % 3) * 3;
        return (r, c);
    }

    public static int[,] DuplicateMatrix(int[,] matrix)
    {
        var m = matrix.GetLength(0);
        var n = matrix.GetLength(1);
        var result = CreateMatrix(m, n);
        for (var i = 0; i < m; ++i)
            for (var j = 0; j < n; ++j)
                result[i, j] = matrix[i, j];
        return result;
    }

    // Cree une matrice aleatoire valide par bloc
    public static int[,] RandomMatrix(Random rnd, int[,] problem)
    {
        var result = DuplicateMatrix(problem);

        for (var block = 0; block < SIZE; ++block)
        {
            var corner = Corner(block);
            var values = Enumerable.Range(1, SIZE).ToList();

            // Melanger les valeurs
            for (var k = 0; k < values.Count; ++k)
            {
                var ri = rnd.Next(k, values.Count);
                (values[k], values[ri]) = (values[ri], values[k]);
            }

            // Retirer les valeurs deja presentes
            var r = corner.row;
            var c = corner.column;
            for (var i = r; i < r + BLOCK_SIZE; ++i)
            {
                for (var j = c; j < c + BLOCK_SIZE; ++j)
                {
                    var value = problem[i, j];
                    if (value != 0)
                        values.Remove(value);
                }
            }

            // Remplir les cellules vides
            var pointer = 0;
            for (var i = r; i < r + BLOCK_SIZE; ++i)
            {
                for (var j = c; j < c + BLOCK_SIZE; ++j)
                {
                    if (result[i, j] == 0)
                    {
                        result[i, j] = values[pointer++];
                    }
                }
            }
        }

        return result;
    }

    // Genere un voisin par echange dans un bloc
    public static int[,] NeighborMatrix(Random rnd, int[,] problem, int[,] matrix)
    {
        var result = DuplicateMatrix(matrix);

        var block = rnd.Next(0, SIZE);
        var corner = Corner(block);
        var cells = new List<int[]>();
        
        for (var i = corner.row; i < corner.row + BLOCK_SIZE; ++i)
        {
            for (var j = corner.column; j < corner.column + BLOCK_SIZE; ++j)
            {
                if (problem[i, j] == 0)
                    cells.Add(new[] { i, j });
            }
        }

        if (cells.Count < 2)
            return result;

        var k1 = rnd.Next(0, cells.Count);
        var inc = rnd.Next(1, cells.Count);
        var k2 = (k1 + inc) % cells.Count;

        var r1 = cells[k1][0];
        var c1 = cells[k1][1];
        var r2 = cells[k2][0];
        var c2 = cells[k2][1];

        (result[r1, c1], result[r2, c2]) = (result[r2, c2], result[r1, c1]);

        return result;
    }

    // Fusionne deux matrices par blocs
    public static int[,] MergeMatrices(Random rnd, int[,] m1, int[,] m2)
    {
        var result = DuplicateMatrix(m1);

        for (var block = 0; block < 9; ++block)
        {
            var pr = rnd.NextDouble();
            if (pr < 0.50)
            {
                var corner = Corner(block);
                for (var i = corner.row; i < corner.row + BLOCK_SIZE; ++i)
                    for (var j = corner.column; j < corner.column + BLOCK_SIZE; ++j)
                        result[i, j] = m2[i, j];
            }
        }

        return result;
    }
}

In [ ]:
// Representation d'une grille Sudoku avec calcul d'erreur
public class SudokuPSO
{
    public int[,] CellValues { get; }

    public SudokuPSO(int[,] cellValues)
    {
        CellValues = MatrixHelperPSO.DuplicateMatrix(cellValues);
    }

    public static SudokuPSO New(int[,] cellValues)
    {
        return new SudokuPSO(MatrixHelperPSO.DuplicateMatrix(cellValues));
    }

    public int Error
    {
        get
        {
            return CountErrors(true) + CountErrors(false);
        }
    }

    private int CountErrors(bool countByRow)
    {
        var errors = 0;
        for (var i = 0; i < MatrixHelperPSO.SIZE; ++i)
        {
            var counts = new int[MatrixHelperPSO.SIZE];
            for (var j = 0; j < MatrixHelperPSO.SIZE; ++j)
            {
                var cellValue = countByRow ? CellValues[i, j] : CellValues[j, i];
                ++counts[cellValue - 1];
            }

            for (var k = 0; k < MatrixHelperPSO.SIZE; ++k)
            {
                if (counts[k] == 0)
                    ++errors;
            }
        }
        return errors;
    }
}

In [ ]:
// Types d'organismes dans l'essaim
public enum OrganismType
{
    Worker,    // Exploite le voisinage
    Explorer   // Explore aleatoirement
}

// Representation d'une particule (organisme)
public class Organism
{
    public OrganismType Type { get; }
    public int[,] Matrix { get; set; }
    public int Error { get; set; }
    public int Age { get; set; }

    public Organism(OrganismType type, int[,] matrix, int error, int age)
    {
        Type = type;
        Error = error;
        Age = age;
        Matrix = MatrixHelperPSO.DuplicateMatrix(matrix);
    }
}

## 3. Implementation du solveur PSO

In [ ]:
using System.Diagnostics;

public class PSOSudokuSolver : ISudokuSolver
{
    private Random _rnd;
    
    // Parametres configurables
    public int NumOrganisms { get; set; } = 200;      // Taille de l'essaim
    public int MaxEpochs { get; set; } = 5000;        // Iterations max
    public int MaxRestarts { get; set; } = 20;        // Redemarrages max
    public double WorkerRatio { get; set; } = 0.90;  // Proportion de workers
    public int MaxAge { get; set; } = 1000;           // Age max avant reinitialisation
    public double MutationRate { get; set; } = 0.001; // Taux de mutation

    public SudokuGrid Solve(SudokuGrid s)
    {
        // Conversion SudokuGrid -> int[,]
        int[,] cellsSolver = new int[9, 9];
        for (int i = 0; i < 9; i++)
            for (int j = 0; j < 9; j++)
                cellsSolver[i, j] = s.Cells[i][j];

        var sudoku = new SudokuPSO(cellsSolver);
        var solvedSudoku = Solve(sudoku);

        // Conversion int[,] -> SudokuGrid
        for (int i = 0; i < 9; i++)
            for (int j = 0; j < 9; j++)
                s.Cells[i][j] = solvedSudoku.CellValues[i, j];

        return s;
    }

    public SudokuPSO Solve(SudokuPSO sudoku)
    {
        var error = int.MaxValue;
        SudokuPSO bestSolution = null;
        var attempt = 0;

        while (error != 0 && attempt < MaxRestarts)
        {
            Console.WriteLine($"Attempt {attempt + 1}/{MaxRestarts}");
            _rnd = new Random(attempt);
            bestSolution = SolveInternal(sudoku);
            error = bestSolution.Error;
            ++attempt;
        }

        return bestSolution;
    }

    private SudokuPSO SolveInternal(SudokuPSO sudoku)
    {
        var numberOfWorkers = (int)(NumOrganisms * WorkerRatio);
        var hive = new Organism[NumOrganisms];

        var bestError = int.MaxValue;
        SudokuPSO bestSolution = null;

        // Initialisation de l'essaim
        for (var i = 0; i < NumOrganisms; ++i)
        {
            var organismType = i < numberOfWorkers ? OrganismType.Worker : OrganismType.Explorer;

            var randomSudoku = SudokuPSO.New(MatrixHelperPSO.RandomMatrix(_rnd, sudoku.CellValues));
            var err = randomSudoku.Error;

            hive[i] = new Organism(organismType, randomSudoku.CellValues, err, 0);

            if (err < bestError)
            {
                bestError = err;
                bestSolution = randomSudoku;
            }
        }

        var epoch = 0;
        while (epoch < MaxEpochs)
        {
            if (epoch % 1000 == 0)
                Console.WriteLine($"  Epoch {epoch}, Best error: {bestError}");

            if (bestError == 0)
                break;

            // Mise a jour de chaque organisme
            for (var i = 0; i < NumOrganisms; ++i)
            {
                if (hive[i].Type == OrganismType.Worker)
                {
                    // Les workers explorent leur voisinage
                    var neighbor = MatrixHelperPSO.NeighborMatrix(_rnd, sudoku.CellValues, hive[i].Matrix);
                    var neighborSudoku = SudokuPSO.New(neighbor);
                    var neighborError = neighborSudoku.Error;

                    var p = _rnd.NextDouble();
                    if (neighborError < hive[i].Error || p < MutationRate)
                    {
                        hive[i].Matrix = MatrixHelperPSO.DuplicateMatrix(neighbor);
                        if (neighborError < hive[i].Error)
                            hive[i].Age = 0;
                        hive[i].Error = neighborError;

                        if (neighborError < bestError)
                        {
                            bestError = neighborError;
                            bestSolution = neighborSudoku;
                        }
                    }
                    else
                    {
                        hive[i].Age++;
                        if (hive[i].Age > MaxAge)
                        {
                            // Reinitialiser le worker stagne
                            var randomSudoku = SudokuPSO.New(MatrixHelperPSO.RandomMatrix(_rnd, sudoku.CellValues));
                            hive[i] = new Organism(OrganismType.Worker, randomSudoku.CellValues, randomSudoku.Error, 0);
                        }
                    }
                }
                else
                {
                    // Les explorers cherchent aleatoirement
                    var randomSudoku = SudokuPSO.New(MatrixHelperPSO.RandomMatrix(_rnd, sudoku.CellValues));
                    hive[i].Matrix = MatrixHelperPSO.DuplicateMatrix(randomSudoku.CellValues);
                    hive[i].Error = randomSudoku.Error;

                    if (hive[i].Error < bestError)
                    {
                        bestError = hive[i].Error;
                        bestSolution = randomSudoku;
                    }
                }
            }

            // Fusion du meilleur worker avec le meilleur explorer
            var bestWorkerIndex = Enumerable.Range(0, numberOfWorkers)
                .OrderBy(i => hive[i].Error).First();
            var bestExplorerIndex = Enumerable.Range(numberOfWorkers, NumOrganisms - numberOfWorkers)
                .OrderBy(i => hive[i].Error).First();
            var worstWorkerIndex = Enumerable.Range(0, numberOfWorkers)
                .OrderByDescending(i => hive[i].Error).First();

            var merged = MatrixHelperPSO.MergeMatrices(_rnd, hive[bestWorkerIndex].Matrix, hive[bestExplorerIndex].Matrix);
            var mergedSudoku = SudokuPSO.New(merged);

            hive[worstWorkerIndex] = new Organism(OrganismType.Worker, merged, mergedSudoku.Error, 0);
            if (hive[worstWorkerIndex].Error < bestError)
            {
                bestError = hive[worstWorkerIndex].Error;
                bestSolution = mergedSudoku;
            }

            ++epoch;
        }

        return bestSolution;
    }
}

## 4. Test du solveur PSO

In [ ]:
// Chargement des puzzles
var puzzles = SudokuHelper.LoadPuzzles("Puzzles/Sudoku_Easy51.txt");
Console.WriteLine($"{puzzles.Count} puzzles charges");

// Test sur un puzzle facile
var puzzle = puzzles[0];
Console.WriteLine("\nPuzzle original:");
SudokuHelper.DisplayGrid(puzzle);

In [ ]:
// Resolution avec PSO
var solver = new PSOSudokuSolver
{
    NumOrganisms = 200,
    MaxEpochs = 3000,
    MaxRestarts = 10
};

var stopwatch = Stopwatch.StartNew();
var solution = solver.Solve(puzzle.Clone());
stopwatch.Stop();

Console.WriteLine($"\nSolution trouvee en {stopwatch.ElapsedMilliseconds}ms:");
SudokuHelper.DisplayGrid(solution);
Console.WriteLine($"\nSolution valide: {SudokuHelper.IsValid(solution)}");

## 5. Benchmark sur plusieurs puzzles

In [ ]:
// Benchmark sur 5 puzzles faciles
var results = new List<(int puzzle, long timeMs, bool success)>();

for (int i = 0; i < Math.Min(5, puzzles.Count); i++)
{
    solver = new PSOSudokuSolver
    {
        NumOrganisms = 200,
        MaxEpochs = 3000,
        MaxRestarts = 10
    };
    
    stopwatch = Stopwatch.StartNew();
    solution = solver.Solve(puzzles[i].Clone());
    stopwatch.Stop();
    
    var isValid = SudokuHelper.IsValid(solution);
    results.Add((i + 1, stopwatch.ElapsedMilliseconds, isValid));
    Console.WriteLine($"Puzzle {i + 1}: {stopwatch.ElapsedMilliseconds}ms - {(isValid ? "Succes" : "Echec")}");
}

Console.WriteLine($"\nResume:");
Console.WriteLine($"  Temps moyen: {results.Average(r => r.timeMs):0}ms");
Console.WriteLine($"  Taux de succes: {results.Count(r => r.success) * 100 / results.Count}%");

## 6. Influence des parametres

### Parametres cles

| Parametre | Effet | Valeur recommandee |
|-----------|-------|-------------------|
| NumOrganisms | Taille de l'essaim | 100-300 |
| MaxEpochs | Iterations par essai | 3000-5000 |
| MaxRestarts | Redemarrages autorises | 10-20 |
| WorkerRatio | Proportion de workers | 0.85-0.95 |
| MaxAge | Age max avant reinit | 500-1000 |

### Comparaison avec GA et SA

| Aspect | PSO | Algorithme Genetique | Recuit Simule |
|--------|-----|---------------------|---------------|
| Inspiration | Oiseaux/poissons | Evolution biologique | Thermodynamique |
| Population | Multiple | Multiple | Unique |
| Exploration | Workers + Explorers | Mutation | Temperature |
| Exploitation | Convergence vers gBest | Crossover | Refroidissement |
| Performance Sudoku | ~1-5s | ~1-10s | ~2-5s |

In [ ]:
// Test avec differents parametres
var configurations = new[]
{
    new { Name = "Conservatif", Organisms = 100, Epochs = 2000, Restarts = 5 },
    new { Name = "Standard", Organisms = 200, Epochs = 3000, Restarts = 10 },
    new { Name = "Agressif", Organisms = 300, Epochs = 5000, Restarts = 20 }
};

var testPuzzle = puzzles[0];

foreach (var config in configurations)
{
    solver = new PSOSudokuSolver
    {
        NumOrganisms = config.Organisms,
        MaxEpochs = config.Epochs,
        MaxRestarts = config.Restarts
    };
    
    stopwatch = Stopwatch.StartNew();
    solution = solver.Solve(testPuzzle.Clone());
    stopwatch.Stop();
    
    Console.WriteLine($"{config.Name}: {stopwatch.ElapsedMilliseconds}ms - {(SudokuHelper.IsValid(solution) ? "Succes" : "Echec")}");
}

## 7. Exercices

### Exercice 1 : Analyse de convergence
Modifiez le solveur pour enregistrer l'evolution de `bestError` au fil des epochs et affichez un graphique de convergence.

### Exercice 2 : Adaptation dynamique
Implementez une adaptation dynamique de `WorkerRatio` qui augmente l'exploration quand la solution stagne.

### Exercice 3 : Hybride PSO-SA
Combinez PSO avec le recuit simule : utilisez SA pour affiner les solutions trouvees par PSO.

## Resume

Le **Particle Swarm Optimization** est une metaheuristique interessante pour Sudoku :

**Avantages** :
- Parallelisation naturelle (essaim)
- Equilibre exploration/exploitation via Workers/Explorers
- Conceptuellement simple

**Inconvenients** :
- Pas de garantie de convergence
- Performance variable selon les puzzles
- Parametres a ajuster

**Quand l'utiliser** :
- Puzzles moyens (pas trop difficiles)
- Quand on veut plusieurs solutions candidates
- Pour comprendre les metaheuristiques d'essaim

---

## Navigation

| << Precedent | [Index](./README.md) | Suivant >> |
|-------------|---------------------|-----------|
| [Sudoku-4-SimulatedAnnealing](Sudoku-4-SimulatedAnnealing.ipynb) | | [Sudoku-6-AIMA-CSP](Sudoku-6-AIMA-CSP.ipynb) |